In [69]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [70]:
root_dir_path = '/home/hwkang/dev-TSB-AD/TSB-AD/results'
postfix = 'dynamic-lnO'

In [71]:
def get_result_files(min_id_code, max_id_code, postfix, k2ex=['encoding', 'loss', 'spikerate']):
    result_file_paths = []
    dir_names = os.listdir(root_dir_path)
    for dir_name in dir_names:
        # if dir_name is in k2ex, continue
        if dir_name in k2ex:
            continue
        file_names = os.listdir(os.path.join(root_dir_path, dir_name))
        for file_name in file_names:
            id = int(file_name.split('_')[0])
            if min_id_code <= id <= max_id_code and postfix in file_name:
                result_file_paths.append(os.path.join(root_dir_path, dir_name, file_name))
    result_file_paths.sort()
    return result_file_paths

In [72]:
result_file_paths = get_result_files(302, 309, postfix)
for result_file_path in result_file_paths:
    print(f'Processing {result_file_path}')
print(f'Total {len(result_file_paths)} files found.')

Processing /home/hwkang/dev-TSB-AD/TSB-AD/results/catsv2/302_ParallelSNN_dynamic_dynamic-lnO.csv
Processing /home/hwkang/dev-TSB-AD/TSB-AD/results/daphnet/303_ParallelSNN_dynamic_dynamic-lnO.csv
Processing /home/hwkang/dev-TSB-AD/TSB-AD/results/gecco/304_ParallelSNN_dynamic_dynamic-lnO.csv
Processing /home/hwkang/dev-TSB-AD/TSB-AD/results/genesis/306_ParallelSNN_dynamic_dynamic-lnO.csv
Processing /home/hwkang/dev-TSB-AD/TSB-AD/results/ghl/305_ParallelSNN_dynamic_dynamic-lnO.csv
Processing /home/hwkang/dev-TSB-AD/TSB-AD/results/ltdb/307_ParallelSNN_dynamic_dynamic-lnO.csv
Processing /home/hwkang/dev-TSB-AD/TSB-AD/results/mitdb/308_ParallelSNN_dynamic_dynamic-lnO.csv
Processing /home/hwkang/dev-TSB-AD/TSB-AD/results/msl/309_ParallelSNN_dynamic_dynamic-lnO.csv
Total 8 files found.


In [73]:
result_file_paths_dgx2 = get_result_files(85, 93, postfix)
for result_file_path in result_file_paths_dgx2:
    print(f'Processing {result_file_path}')
print(f'Total {len(result_file_paths_dgx2)} files found.')

Processing /home/hwkang/dev-TSB-AD/TSB-AD/results/dgx2/085_ParallelSNN_dynamic_dynamic-lnO.csv
Processing /home/hwkang/dev-TSB-AD/TSB-AD/results/dgx2/086_ParallelSNN_dynamic_dynamic-lnO.csv
Processing /home/hwkang/dev-TSB-AD/TSB-AD/results/dgx2/087_ParallelSNN_dynamic_dynamic-lnO.csv
Processing /home/hwkang/dev-TSB-AD/TSB-AD/results/dgx2/088_ParallelSNN_dynamic_dynamic-lnO.csv
Processing /home/hwkang/dev-TSB-AD/TSB-AD/results/dgx2/089_ParallelSNN_dynamic_dynamic-lnO.csv
Processing /home/hwkang/dev-TSB-AD/TSB-AD/results/dgx2/090_ParallelSNN_dynamic_dynamic-lnO.csv
Processing /home/hwkang/dev-TSB-AD/TSB-AD/results/dgx2/091_ParallelSNN_dynamic_dynamic-lnO.csv
Processing /home/hwkang/dev-TSB-AD/TSB-AD/results/dgx2/092_ParallelSNN_dynamic_dynamic-lnO.csv
Processing /home/hwkang/dev-TSB-AD/TSB-AD/results/dgx2/093_ParallelSNN_dynamic_dynamic-lnO.csv
Total 9 files found.


In [74]:
result_file_paths = result_file_paths + result_file_paths_dgx2
result_file_paths.sort()
print(f'Total {len(result_file_paths)} files after merging.')

Total 17 files after merging.


In [75]:
# merge every records in every result file
def merge_records(result_file_paths):
    records = []
    for result_file_path in result_file_paths:
        df = pd.read_csv(result_file_path)
        records.append(df)
    merged_df = pd.concat(records, ignore_index=True)
    # sort by 'file' header
    merged_df.sort_values(by='file', inplace=True)
    return merged_df

In [76]:
df = merge_records(result_file_paths)

In [77]:
log_dir_path = '/home/hwkang/dev-TSB-AD/TSB-AD/logs'

# get last id code from the file name (format is like '299_ParallelSNN_dynamic_dynamic-ln.csv')
log_file_names = os.listdir(log_dir_path)
# sort descending order
log_file_names.sort(reverse=True)
last_id_code = int(log_file_names[0].split('_')[0])

# assert number of records is exactly equal to 180
assert len(df) == 180, f'Expected 180 records, but got {len(df)} records.'

latest_id_code = last_id_code + 1
# create dummy log file with latest_id_code in log_dir_path
dummy_log_file_path = os.path.join(log_dir_path, f'{latest_id_code}_ParallelSNN_dynamic_dynamic-ln.log')
with open(dummy_log_file_path, 'w') as f:
    f.write('This is a dummy log file for the latest id code.\n')
    f.write(f'Latest id code: {latest_id_code}\n')

# save df to 
target_tokens = result_file_paths[0].split('/')[-1].split('_')[1:]
target_file_name = '_'.join(target_tokens)
#print(f'Target file name: {target_file_name}')
dst_file_path = os.path.join('/home/hwkang/dev-TSB-AD/TSB-AD/results/full', f'{latest_id_code}_{target_file_name}')
#print(f'Saving merged records to {dst_file_path}')

df.to_csv(dst_file_path, index=False)